In [1]:
# import libraries
from pyspark.context import SparkContext,SparkConf
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as F
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

## Assignments

#### 1)What are the lowest and highest temperatures measured each year for the period 1950-2014. Provide the lists sorted in the descending order with respect to the maximum temperature. In this exercise you will use the temperature-readings.csv file.¶

In [2]:
temp_csv = spark.read.csv("temperature-readings-tiny.csv", header=False,sep=";")
df=temp_csv.selectExpr("_c0 as Station_number", "_c1 as Date", "_c2 as Time", "_c3 as Temperature","_c4 as Quality")
df=df.withColumn('Date',F.to_timestamp("Date", "yyyy-MM-dd"))
df=df.orderBy(df.Temperature,ascending=False)
df.show()
df.printSchema()

+--------------+-------------------+--------+-----------+-------+
|Station_number|               Date|    Time|Temperature|Quality|
+--------------+-------------------+--------+-----------+-------+
|        102170|2016-06-11 00:00:00|06:00:00|        9.9|      G|
|        102190|1955-09-15 00:00:00|12:00:00|        9.9|      Y|
|        102190|1957-05-02 00:00:00|12:00:00|        9.9|      Y|
|        102190|1956-09-12 00:00:00|00:00:00|        9.9|      Y|
|        102190|1957-06-23 00:00:00|12:00:00|        9.9|      Y|
|        102190|1960-06-17 00:00:00|06:00:00|        9.9|      Y|
|        102170|2014-09-21 00:00:00|18:00:00|        9.9|      G|
|        102170|2015-05-26 00:00:00|18:00:00|        9.9|      G|
|        102170|2014-10-08 00:00:00|06:00:00|        9.9|      G|
|        102170|2015-06-17 00:00:00|06:00:00|        9.9|      G|
|        102190|1956-08-25 00:00:00|00:00:00|        9.9|      Y|
|        102190|1956-09-05 00:00:00|00:00:00|        9.9|      Y|
|        1

In [8]:
# maximun temperature per year
df.createOrReplaceTempView("temps_table")
query_max_temps="SELECT YEAR(Date) as year, MAX(FLOAT(Temperature)) AS max_Temperature \
FROM temps_table WHERE YEAR(Date) >=1950 AND YEAR(Date) <= 2014 GROUP BY year ORDER BY max_Temperature DESC"

max_temps=spark.sql(query_max_temps)
max_temps.show()
max_temps.printSchema()

+----+---------------+
|year|max_Temperature|
+----+---------------+
|2014|           29.1|
|1960|           29.0|
|1959|           28.2|
|1958|           28.1|
|1956|           26.0|
|1957|           25.2|
|1955|           20.4|
|1961|           19.0|
|2013|           10.2|
+----+---------------+

root
 |-- year: integer (nullable = true)
 |-- max_Temperature: float (nullable = true)



In [9]:
# min temperatures per year
query_min_temps="SELECT YEAR(Date) as year, MIN(FLOAT(Temperature)) AS min_Temperature \
FROM temps_table WHERE YEAR(Date) >=1950 AND YEAR(Date) <= 2014 GROUP BY year ORDER BY min_Temperature ASC"

min_temps=spark.sql(query_min_temps)
min_temps.show()
min_temps.printSchema()

+----+---------------+
|year|min_Temperature|
+----+---------------+
|1956|          -30.0|
|1960|          -28.3|
|1958|          -27.9|
|1955|          -26.2|
|2014|          -24.3|
|1961|          -23.5|
|1959|          -23.2|
|1957|          -19.9|
|2013|          -13.3|
+----+---------------+

root
 |-- year: integer (nullable = true)
 |-- min_Temperature: float (nullable = true)



### Question 1.a

In [10]:
# max temperature for every station per year
query_max_temps_station="SELECT YEAR(Date) as year, Station_number,MAX(FLOAT(Temperature)) AS max_Temperature \
FROM temps_table WHERE YEAR(Date) >=1950 AND YEAR(Date) <= 2014 GROUP BY year ,Station_number ORDER BY max_Temperature DESC"

max_temps_station=spark.sql(query_max_temps_station)
max_temps_station.show()

+----+--------------+---------------+
|year|Station_number|max_Temperature|
+----+--------------+---------------+
|2014|        102170|           29.1|
|1960|        102190|           29.0|
|1959|        102190|           28.2|
|1958|        102190|           28.1|
|1956|        102190|           26.0|
|1957|        102190|           25.2|
|1955|        102190|           20.4|
|1961|        102190|           19.0|
|2013|        102170|           10.2|
+----+--------------+---------------+



In [11]:
# min temperature for every station per year
query_min_temps_station="SELECT YEAR(Date) as year, Station_number,MIN(FLOAT(Temperature)) AS min_Temperature \
FROM temps_table WHERE YEAR(Date) >=1950 AND YEAR(Date) <= 2014 GROUP BY year ,Station_number ORDER BY min_Temperature ASC"

min_temps_station=spark.sql(query_min_temps_station)
min_temps_station.show()

+----+--------------+---------------+
|year|Station_number|min_Temperature|
+----+--------------+---------------+
|1956|        102190|          -30.0|
|1960|        102190|          -28.3|
|1958|        102190|          -27.9|
|1955|        102190|          -26.2|
|2014|        102170|          -24.3|
|1961|        102190|          -23.5|
|1959|        102190|          -23.2|
|1957|        102190|          -19.9|
|2013|        102170|          -13.3|
+----+--------------+---------------+



### Question 1.b

In [58]:
# see Lab 2 ===> solution with pandas

### Question 2

#### 2)Count the number of readings for each month in the period of 1950-2014 which are higher than 10 degrees.Repeat the exercise,this time taking only distinct readings from each station. That is, if a station reported are a ding above 10 degrees in some month,then it appears only once in the count for that month.In this exercise you will use the temperature-readings.csvfile.¶
#### The out put should contain the following information:Year, month, count

In [12]:
# using the temps_table from before

query_over10=" SELECT YEAR(Date) AS year , MONTH(Date) AS month ,COUNT(Temperature) AS count \
FROM temps_table WHERE FLOAT(Temperature) >=10 AND YEAR(Date) >=1950 AND YEAR(Date) <= 2014  GROUP BY year,month ORDER BY year,month ASC"

spark.sql(query_over10).show()

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|1955|    9|   55|
|1955|   10|   23|
|1956|    3|    4|
|1956|    4|    7|
|1956|    5|   60|
|1956|    6|   92|
|1956|    7|  108|
|1956|    8|   84|
|1956|    9|   54|
|1956|   10|   17|
|1957|    3|    1|
|1957|    4|   13|
|1957|    5|   46|
|1957|    6|   72|
|1957|    7|  109|
|1957|    8|   96|
|1957|    9|   43|
|1957|   10|    9|
|1958|    4|    4|
|1958|    5|   40|
+----+-----+-----+
only showing top 20 rows



In [14]:
# now using distinct temperatures

query_over10_distinct=" SELECT YEAR(Date) AS year , MONTH(Date) AS month ,AVG(DISTINCT(Temperature)) AS count \
FROM temps_table WHERE FLOAT(Temperature) >=10 AND YEAR(Date) >=1950 AND YEAR(Date) <= 2014 GROUP BY year,month ORDER BY year,month ASC"

spark.sql(query_over10_distinct).show()

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|1955|    9|   35|
|1955|   10|   20|
|1956|    3|    4|
|1956|    4|    5|
|1956|    5|   44|
|1956|    6|   58|
|1956|    7|   76|
|1956|    8|   49|
|1956|    9|   30|
|1956|   10|   13|
|1957|    3|    1|
|1957|    4|   13|
|1957|    5|   33|
|1957|    6|   51|
|1957|    7|   69|
|1957|    8|   58|
|1957|    9|   27|
|1957|   10|    8|
|1958|    4|    3|
|1958|    5|   31|
+----+-----+-----+
only showing top 20 rows



### Question 3

#### 3)Find the average monthly temperature for each available station in Sweden. Your result should include average temperature for each station for each month in the period of 1960-2014. Bear in mind that not every station has the readings for each month in this timeframe.In this exercise you will use the temperature-readings.csvfile.¶
#### The output should contain the following information: Year, month, station number, average monthly temperature


### The monthly average is calculated by averaging the daily maximums and minimums for that month.For example, to get the monthly average for October, take maximums and minimums for each day, sum them up and divide by 62 (which is the same as taking the daily averages, summing them up and divide by the number of days).

In [39]:
query=" WITH sub AS (SELECT YEAR(Date) AS year , MONTH(Date) AS month , DAY(Date) AS day,Station_number AS station,AVG(FLOAT(Temperature)) AS avg_daily \
FROM temps_table WHERE YEAR(Date) >=1950 AND YEAR(Date) <= 2014 GROUP BY year,month,day,station ORDER BY (year,month,day) DESC) \
SELECT year,month,station,AVG(avg_daily) AS avg_monthly FROM sub GROUP BY year,month,station \
ORDER BY (year,month,station) DESC"

spark.sql(query).show()



+----+-----+-------+--------------------+
|year|month|station|         avg_monthly|
+----+-----+-------+--------------------+
|2014|   12| 102170|  -5.801612894020734|
|2014|   11| 102170|  2.5249999977648256|
|2014|   10| 102170|   7.106451625304837|
|2014|    9| 102170|   8.584999987483025|
|2014|    8| 102170|   13.86935483255694|
|2014|    7| 102170|    19.6596774132021|
|2014|    6| 102170|  14.443333387374878|
|2014|    5| 102170|  10.756451619728919|
|2014|    4| 102170|   4.776666717727979|
|2014|    3| 102170|  1.8967741969371996|
|2014|    2| 102170|  0.6749999944918922|
|2014|    1| 102170|  -4.106451601751389|
|2013|   12| 102170|  0.7096774189943268|
|2013|   11| 102170|-0.05166668320695559|
|1961|    5| 102190|  10.615333366394044|
|1961|    4| 102190|   4.507777822679944|
|1961|    3| 102190|  2.7086021617375398|
|1961|    2| 102190|  -2.133333349955224|
|1961|    1| 102190|  -7.215053754468119|
|1960|   12| 102190| -2.4274193498115704|
+----+-----+-------+--------------